In [7]:
# Author Kate Lisovenko

import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns


The parameter **na_values** = ["NA", "N/A", "-", "", " "] is used when loading data in pandas to specify certain values that should be interpreted as missing data (NaN).<br>
This parameter creates a list of values that pandas will recognize as placeholders for missing or null entries.


In [8]:
# loading data
na_values = ["NA", "N/A", "-", "", " "] # t
data = pd.read_csv('././weather_6_data.csv',na_values=na_values )
data['date'].head(3)

0    10-apr-1996 14:00
1    31-jul-1996 08:00
2    31-jul-1996 09:00
Name: date, dtype: object

In [9]:
#%d for the two-digit day
#%b for the abbreviated month name
#%Y for the four-digit year

data['date'] = pd.to_datetime(data['date'],format='%d-%b-%Y %H:%M')
#data['date'].head()

In [10]:
print(f"The first date of observation is {data['date'].min()}")
print(f"The last date of observation is {data['date'].max()}")

The first date of observation is 1996-04-10 14:00:00
The last date of observation is 2024-10-01 00:00:00


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246930 entries, 0 to 246929
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    246930 non-null  datetime64[ns]
 1   ind     246930 non-null  int64         
 2   rain    246881 non-null  float64       
 3   ind.1   246930 non-null  int64         
 4   temp    246930 non-null  float64       
 5   ind.2   246930 non-null  int64         
 6   wetb    246930 non-null  float64       
 7   dewpt   246930 non-null  float64       
 8   vappr   246930 non-null  float64       
 9   rhum    246930 non-null  int64         
 10  msl     246912 non-null  float64       
 11  ind.3   246930 non-null  int64         
 12  wdsp    246880 non-null  float64       
 13  ind.4   246930 non-null  int64         
 14  wddir   246878 non-null  float64       
 15  ww      192218 non-null  float64       
 16  w       192217 non-null  float64       
 17  sun     246930 non-null  floa

In [12]:
# searching for missing data
missing_percentage = round(data.isnull().sum() / len(data) * 100, 2)
missing_percentage

date      0.00
ind       0.00
rain      0.02
ind.1     0.00
temp      0.00
ind.2     0.00
wetb      0.00
dewpt     0.00
vappr     0.00
rhum      0.00
msl       0.01
ind.3     0.00
wdsp      0.02
ind.4     0.00
wddir     0.02
ww       22.16
w        22.16
sun       0.00
vis      25.03
clht      0.01
clamt     0.01
dtype: float64

##### There is a sagnificant lack of data here:
ww:    -  Synop code for Present Weather -22.16%<br>
w:     -  Synop code for Past Weather -22.16%<br>
vis:   -  Visibility (m) - 25.03%<br>

### The Temperature plot

### The mean temperature each day

### The mean temperature for each month

### The Windspeed (there is data missing from this column)

### The rolling windspeed (say over 24 hours)

### The max windspeed for each day

### The monthly mean of the daily max windspeeds (yer I am being nasty here)